In [1]:
from data.data_pipelines import etl_dowaward_giordano

In [2]:
%load_ext autoreload
%autoreload 2

In [21]:
df = etl_dowaward_giordano()

querying database.. 
2024-03-24 22:39:43,745 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-24 22:39:43,745 INFO sqlalchemy.engine.Engine SELECT stock_prices.id, stock_prices.ticker, stock_prices.price_date, stock_prices.open, stock_prices.close, stock_prices.adjusted_close, stock_prices.high, stock_prices.low, stock_prices.volume, stock_prices.data_source, stock_prices.update_date 
FROM stock_prices
2024-03-24 22:39:43,746 INFO sqlalchemy.engine.Engine [generated in 0.00068s] ()
2024-03-24 22:39:44,002 INFO sqlalchemy.engine.Engine ROLLBACK
calculating returns..
calculating garch..


In [7]:
import pandas as pd
pd.set_option('display.max_rows', 500)

correlation                                       ...  \
ticker             AGG DBC EEM EFA IGOV IJH IJR RWR SHY VAW  ...   
price_date                                                   ...   
2000-05-26         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   
2000-05-30         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   
2000-05-31         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   
2000-06-01         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   
2000-06-02         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   
...                ...  ..  ..  ..  ...  ..  ..  ..  ..  ..  ...   
2024-03-18         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   
2024-03-19         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   
2024-03-20         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   
2024-03-21         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   
2024-03-22         NaN NaN NaN NaN  NaN NaN NaN NaN NaN NaN  ...   

           adjusted_daily_returns                                          \
ticker                        DBC       EEM       EFA      IGOV       IJH   
price_date                                                                  
2000-05-26                    NaN       NaN       NaN       NaN       NaN   
2000-05-30                    NaN       NaN       NaN       NaN  0.038987   
2000-05-31                    NaN       NaN       NaN       NaN -0.001645   
2000-06-01                    NaN       NaN       NaN       NaN  0.018133   
2000-06-02                    NaN       NaN       NaN       NaN  0.038860   
...                           ...       ...       ...       ...       ...   
2024-03-18               0.009227  0.001717 -0.000253 -0.002511 -0.002223   
2024-03-19              -0.000435 -0.004897  0.001775 -0.001762  0.007885   
2024-03-20              -0.002613  0.011319  0.010632  0.006053  0.013605   
2024-03-21              -0.001310  0.001217 -0.000501 -0.003510  0.010767   
2024-03-22              -0.006996 -0.007047 -0.002130 -0.000503 -0.007989   

                                                              
ticker           IJR       RWR       SHY       VAW        VV  
price_date                                                    
2000-05-26       NaN       NaN       NaN       NaN       NaN  
2000-05-30  0.019921       NaN       NaN       NaN       NaN  
2000-05-31  0.019532       NaN       NaN       NaN       NaN  
2000-06-01  0.023241       NaN       NaN       NaN       NaN  
2000-06-02  0.028545       NaN       NaN       NaN       NaN  
...              ...       ...       ...       ...       ...  
2024-03-18 -0.003205  0.000391  0.000123  0.000201  0.006177  
2024-03-19  0.006904 -0.000217  0.000859  0.002366  0.004615  
2024-03-20  0.016532  0.006300  0.001594  0.011297  0.010241  
2024-03-21  0.011641  0.006477  0.000000  0.005064  0.003212  
2024-03-22 -0.012643 -0.013942  0.000734 -0.009831 -0.004865  

[5993 rows x 22 columns]

In [ ]:



def calculate_ATR(df):
    pass

def rank_signals(df, columns_to_rank):
    pass